#Quiz 5 (Take Home) Assignment

## Installing Neccesary Libraries


In [ ]:
!pip install tensorflow
!pip install keras
!pip install nltk

In [3]:
from google.colab import drive
import os
import zipfile
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import numpy as np
import pickle


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#Check for the file I need
directory_path = '/content/drive/My Drive/Projects Things Useful/University/CU Denver 2023-2024 Sem 1/Deep Learning/Quiz_5_Data'
pickle_path = directory_path + "/Pickle_Files"
# Check if the directory exists
if os.path.exists(directory_path):
    # List all files and directories in the specified path
    files = os.listdir(directory_path)
    print("Files and directories in '", directory_path, "' :")
    for i in files:
      print(i)
else:
    print("The directory does not exist")

Files and directories in ' /content/drive/My Drive/Projects Things Useful/University/CU Denver 2023-2024 Sem 1/Deep Learning/Quiz_5_Data ' :
training-set-100000.json
test-set-20000-rev2.json
judge-set-10000-unannotated.json
Pickle_Files
BioWordVec_PubMed_MIMICIII_d200.vec.bin


In [5]:
import json
file_name= "/training-set-100000.json"
training_path = f'{directory_path}/{file_name}'
data = []
with open(training_path, 'r') as file:
    data = json.load(file)
    if data:
      print("Training Data Loaded")

Training Data Loaded


## Step 1. Data Pre-Processing

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Download stopwords and wordnet data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Load English stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove HTML tags using regex
    text = re.sub(r'<.*?>', '', text)

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Lowercase all texts
    text = text.lower()

    # Tokenize text
    words = text.split()

    # Remove stopwords and apply stemming or lemmatization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # Uncomment the next line to use stemming instead of lemmatization
    # words = [stemmer.stem(word) for word in words if word not in stop_words]

    # Join the words back into a single string
    text = ' '.join(words)

    return text

# cleaned_texts = [clean_text(text) for text in texts]

# # Now `cleaned_texts` contains the preprocessed article texts


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# Assuming `data` is already loaded with the articles data in the above format
# Combine title and abstract for each article and gather labels
texts = [clean_text(d['title'] + ' ' + d['abstractText']) for d in data['articles']]
labels = [d['meshMajor'] for d in data['articles']]

# Initialize the tokenizer with a specific vocabulary size, and a filter for punctuation
vocabulary_size = 10000  # for example, you might choose the top 20,000 words
tokenizer = Tokenizer(num_words=vocabulary_size, lower=True, oov_token='UNK', filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

tokenizer.fit_on_texts(texts)

# Convert texts to sequences of integer indices
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

# Determine a suitable maximum sequence length based on the data distribution
# For instance, you can take the 90th percentile as the max length
max_seq_length = int(np.percentile([len(seq) for seq in sequences], 50))

# Pad the sequences so that they all have the same length
data_padded = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

# Initialize MultiLabelBinarizer to convert the labels to a binary matrix
mlb = MultiLabelBinarizer()
label_data = mlb.fit_transform(labels)

# This absolutely devours material.
# # Save the tokenizer, MultiLabelBinarizer, max_seq_length for later use
# with open(pickle_path + '/preprocessing_objects.pickle', 'wb') as handle:
#     pickle.dump({'tokenizer': tokenizer, 'mlb': mlb, 'max_seq_length': max_seq_length,
#                  'data_padded': data_padded, 'label_data': label_data}, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Now `data_padded` contains the padded sequences and `label_data` contains the multi-hot encoded labels
# `word_index` contains the word index, and `max_seq_length` is the length up to which sequences will be padded
print("Preprocessing and saving completed.")


Preprocessing and saving completed.


In [ ]:
# # Load all preprocessing objects at once
# with open(pickle_path + '/preprocessing_objects.pickle', 'rb') as handle:
#     preprocessing_objects = pickle.load(handle)

# tokenizer = preprocessing_objects['tokenizer']
# mlb = preprocessing_objects['mlb']
# max_seq_length = preprocessing_objects['max_seq_length']
# # data_padded = preprocessing_objects['data_padded']
# label_data = preprocessing_objects['label_data']
# print("Loaded")

Loaded


In [ ]:
n = 10
for i, (word, index) in enumerate(tokenizer.word_index.items()):
    print(f"{i + 1}. {word}: {index}")
    if i >= n - 1:  # since index starts at 0, we use n - 1
        break

1. UNK: 1
2. patient: 2
3. cell: 3
4. study: 4
5. result: 5
6. protein: 6
7. p: 7
8. group: 8
9. effect: 9
10. level: 10


## Step 2. Creating the Embedded Layer 🐪

### RUN THIS ONCE, AFTERWARDS JUST USE PICKLE FILE VERSION
(IT TAKES FOREVER TO LOAD)

In [5]:
from gensim.models import KeyedVectors

embedding_path = directory_path + "/BioWordVec_PubMed_MIMICIII_d200.vec.bin"
embeddings = KeyedVectors.load_word2vec_format(embedding_path, binary=True)



# Get the word_index from your tokenizer
word_index = tokenizer.word_index

# Initialize the embedding matrix
embedding_matrix = np.zeros((len(word_index) + 1, 200))  # Add 1 for padding token

# Populate the embedding matrix
for word, i in word_index.items():
    # Check if the word is in the model
    if word in embeddings.key_to_index:
        # Get the embedding vector for the word
        embedding_vector = embeddings[word]
        # If an embedding was found, add it to the matrix
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

with open(pickle_path + "/embedding_matrix_10000", 'wb') as handle:
    pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Embedding matrix saved.")


KeyboardInterrupt: ignored

In [7]:
# Create the embedding layer with pre-trained weights

loaded_embedding_matrix = []
with open(pickle_path + "/embedding_matrix_10000", 'rb') as handle:
    loaded_embedding_matrix = pickle.load(handle)

embedding_layer = Embedding(input_dim=len(word_index) + 1,
                            output_dim=200,
                            weights=[loaded_embedding_matrix],
                            input_length=max_seq_length,  # As determined earlier
                            trainable=False)

## Step 3. Create LSTM layers

In [9]:
# Number of unique labels in your multi-label classification task
num_labels = len(mlb.classes_)  # Replace with actual number of labels

model = Sequential()
# Add the pre-loaded embedding layer
model.add(embedding_layer)  # embedding_layer is the one you created using loaded_embedding_matrix
# Add an LSTM layer
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=64))
model.add(Dropout(0.2))
# Add a dense layer
# You can adjust the number of units and add more dense layers if needed
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.2))
# Output layer with a sigmoid activation for multi-label classification
model.add(Dense(units=num_labels, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 123, 200)          54306400  
                                                                 
 lstm (LSTM)                 (None, 123, 128)          168448    
                                                                 
 dropout (Dropout)           (None, 123, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0

## Step 4. Train the model and pray that Google Colab Pro is Enough!

In [14]:
# Load the training data
X_train = data_padded  # Your padded sequence data
Y_train = label_data   # Your multi-hot encoded MeSH terms

In [10]:
from tensorflow.keras.callbacks import EarlyStopping



# Set training parameters
batch_size = 32
epochs = 25
validation_split = 0.1  # Percentage of data to use as validation

# Define an EarlyStopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Train the model
history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=validation_split,
                    callbacks=[early_stopping],
                    verbose=1)

print("Training completed.")

Epoch 1/25
2813/2813 [==============================] - 409s 144ms/step - loss: 0.0088 - accuracy: 0.0047 - val_loss: 0.0036 - val_accuracy: 0.0053
Epoch 2/25
2813/2813 [==============================] - 403s 143ms/step - loss: 0.0037 - accuracy: 0.0047 - val_loss: 0.0036 - val_accuracy: 0.0053
Epoch 3/25
2813/2813 [==============================] - 403s 143ms/step - loss: 0.0037 - accuracy: 0.0047 - val_loss: 0.0036 - val_accuracy: 0.0053
Epoch 4/25
2813/2813 [==============================] - 410s 146ms/step - loss: 0.0036 - accuracy: 0.0864 - val_loss: 0.0034 - val_accuracy: 0.0887
Epoch 5/25
2813/2813 [==============================] - 406s 144ms/step - loss: 0.0034 - accuracy: 0.1435 - val_loss: 0.0033 - val_accuracy: 0.1789
Epoch 6/25
2813/2813 [==============================] - 408s 145ms/step - loss: 0.0032 - accuracy: 0.1452 - val_loss: 0.0031 - val_accuracy: 0.1603
Epoch 7/25
2813/2813 [==============================] - 405s 144ms/step - loss: 0.0031 - accuracy: 0.1358 - val_

In [8]:
model_save_path = pickle_path + '/saved_model_25_Epochs.h5'  # Replace with your desired path


In [11]:
# Save the model
# V1 of model
# model_save_path = pickle_path + '/saved_model.h5'  # Replace with your desired path
model_save_path = pickle_path + '/saved_model_25_Epochs.h5'  # Replace with your desired path
model.save(model_save_path)
print(f"Model saved at {model_save_path}")


Model saved at /content/drive/My Drive/Projects Things Useful/University/CU Denver 2023-2024 Sem 1/Deep Learning/Quiz_5_Data/Pickle_Files/saved_model_25_Epochs.h5


## Step 5. Call the saved model and run it.

In [10]:
from tensorflow.keras.models import load_model

# Load the model
loaded_model = load_model(model_save_path)
print("Model loaded successfully.")

Model loaded successfully.


In [15]:
first_sample = X_train[1:2]  # Select the first sample
predicted_result = loaded_model.predict(first_sample)  # Predict using the loaded model

# Print the predicted result
print("Predicted result:", predicted_result)

# Print the actual result
actual_result = Y_train[1]
print("Actual result:", actual_result)



1/1 [==============================] - 0s 45ms/step
Predicted result: [[2.0347984e-07 5.3732501e-06 1.3278274e-06 ... 4.8155474e-05
  9.4666415e-05 1.7087568e-05]]
Actual result: [0 0 0 ... 0 0 0]


In [16]:
# Use inverse_transform to convert binary vectors back to labels
import numpy as np

# Convert predicted result to binary and then to labels
predicted_labels_conformed = mlb.inverse_transform(np.round(predicted_result))

# Ensure actual_result is a 2D array for inverse_transform
actual_labels_conformed = mlb.inverse_transform(np.array([actual_result]))

print("Predicted MeSH terms:", predicted_labels_conformed[0])
print("Actual MeSH terms:", actual_labels_conformed[0])



Predicted MeSH terms: ('Aged', 'Female', 'Humans', 'Male', 'Middle Aged')
Actual MeSH terms: ('Adult', 'Aged', 'Aged, 80 and over', 'Carcinoma, Squamous Cell', 'Case-Control Studies', 'Female', 'Gene Frequency', 'Genetic Predisposition to Disease', 'Genotype', 'Germany', 'Greece', 'Humans', 'Interleukin-8', 'Male', 'Middle Aged', 'Mouth Neoplasms', 'Polymorphism, Genetic', 'Reverse Transcriptase Polymerase Chain Reaction', 'Risk')


Step 4. Seeing how well we did :3

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, hamming_loss, jaccard_score, roc_auc_score
import numpy as np

X_train = data_padded
Y_train = label_data

# Make predictions for the entire training set
train_predictions = loaded_model.predict(X_train)

# Convert predictions to binary format
binary_predictions = np.round(train_predictions)

# Calculate metrics
recall = recall_score(Y_train, binary_predictions, average='micro')
# f1 = f1_score(Y_train, binary_predictions, average='micro')
# hamming = hamming_loss(Y_train, binary_predictions)
# jaccard = jaccard_score(Y_train, binary_predictions, average='micro')
# roc_auc = roc_auc_score(Y_train, train_predictions, average='micro')

print("Recall on Training Data:", recall)
# print("F1 Score on Training Data:", f1)
# print("Hamming Loss on Training Data:", hamming)
# print("Jaccard Index on Training Data:", jaccard)
# print("ROC-AUC on Training Data:", roc_auc)


3125/3125 [==============================] - 160s 51ms/step


In [21]:
print(Y_train[0])
print(binary_predictions[1])

predicted_labels_conformed = mlb.inverse_transform(np.round(binary_predictions))
actual_labels_conformed = mlb.inverse_transform(np.array([Y_train]))



[0 0 0 ... 0 0 0]
[0. 0. 0. ... 0. 0. 0.]


ValueError: ignored

In [20]:

print(predicted_labels_conformed)
print(actual)

[('Aged', 'Female', 'Humans', 'Male', 'Middle Aged'), ('Animals', 'Rats'), ('Female', 'Humans', 'Male'), (), ('Adult', 'Aged', 'Blood Pressure', 'Female', 'Humans', 'Hypertension', 'Male', 'Middle Aged'), ('Animals',), ('Animals', 'Humans'), ('Humans', 'Male'), ('Female', 'Humans', 'Male'), ('Humans',), ('Humans',), ('Animals', 'Mice'), ('Animals',), ('Software',), ('Animals',), ('Humans', 'Male'), ('Adult', 'Female', 'Humans', 'Male'), ('Animals', 'Mice'), ('Animals', 'Female', 'Male'), ('Humans',), (), (), ('Female', 'Humans', 'Male'), ('Adult', 'Female', 'Humans', 'Male', 'Middle Aged'), ('Humans',), (), (), ('Animals', 'Male'), ('Animals',), ('Adult', 'Female', 'Humans', 'Male'), ('Aged', 'Female', 'Humans', 'Middle Aged'), ('Animals', 'Base Sequence', 'Mice'), ('Animals',), ('Animals',), (), ('Animals', 'Mice'), ('Cell Line, Tumor', 'Humans'), ('Adult', 'Female', 'Humans', 'Male'), (), ('Animals', 'Mice'), ('Female', 'Humans', 'Male'), ('Adult', 'Female', 'Humans', 'Male'), (), ('